In [ ]:
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'   # Use GPU if available, otherwise CPU

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [ ]:
# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define fully connected layers
        self.l1 = nn.Linear(784, 520)  # Input layer (28x28=784) to hidden layer
        self.l2 = nn.Linear(520, 320)  # Hidden layer
        self.l3 = nn.Linear(320, 240)  # Hidden layer
        self.l4 = nn.Linear(240, 120)  # Hidden layer
        self.l5 = nn.Linear(120, 10)   # Output layer (10 classes)


    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))  # Apply ReLU activation
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)       # Output logits

In [ ]:
# Initialize model, loss function, and optimizer
model = Net()
model.to(device) # Move model to the appropriate device (CPU/GPU)
criterion = nn.CrossEntropyLoss()   # Loss function for multi-class classification
optimizer = torch.optim.Adam(model.parameters(),    # Adam optimizer
                             lr=0.01, 
                             betas=(0.9, 0.999),
                             eps=1e-08,
                             weight_decay=0,
                             amsgrad=False)

# Training function
def train(epoch):
    model.train()   # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data and target to the device
        data, target = data.to(device), target.to(device)   
        optimizer.zero_grad()               # Zero the gradients
        output = model(data)                # Forward pass
        loss = criterion(output, target)    # Compute loss
        loss.backward()                     # Backward pass
        optimizer.step()                    # Update weights
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# Testing function
def test():
    model.eval()    # Set model to evaluation mode
    test_loss = 0   # Initialize test loss
    correct = 0     # Initialize correct predictions counter
    for data, target in test_loader:
        # Move data and target to the device
        data, target = data.to(device), target.to(device)
        output = model(data)                            # Forward pass
        test_loss += criterion(output, target).item()   # Accumulate loss

        # Get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]      
        # Count correct predictions
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    # Average the test loss
    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [ ]:
# Main function
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.304297
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 1.670444
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 0.762810
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 0.877331
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 0.819677
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 0.368655
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 0.649755
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 0.185701
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 0.272089
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 0.622051
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 0.344957
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 0.407387
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 0.273092
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 0.321562
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 0.682199
Train Epoch: 1 | Batch Status: 9600/60